In [113]:
#### import global modules
import os
import sys
import pandas as pd
import numpy as np
from pathlib import Path
from yaml import safe_load
import google.oauth2.credentials
from google.cloud import bigquery
import gc

# Set global vars
pth_project = Path(os.getcwd().split('notebooks')[0])
pth_data = pth_project / 'data'
pth_queries = pth_project / 'core' / 'queries'
pth_creds = pth_project / 'conf' / 'local' / 'project_config.yaml'
sys.path.insert(0, str(pth_project))
d_project_config = safe_load(pth_creds.open())
# d_params = safe_load((pth_project / 'core' / 'parameters' / 'common.yaml').open())['data_extract']

# import local modules
from core.utils.gcp import connect_bq_services
# from core.etl.extract import extract_bq_data, extract_pr_codes, format_conv_df, filter_convs

# Connect to google services
bq_client = connect_bq_services(d_project_config['gcp-project-name'])
pd.options.display.max_rows = 100

In [114]:
def extract_bq_data(bq_client, sql=None, pth_query=None):
    if sql is not None:
        df = bq_client.query(sql).to_dataframe()
    elif pth_query is not None:
        sql = pth_query.read_text()
        df = bq_client.query(sql).to_dataframe()
    else:
        raise ValueError('`sql` or `pth_query` should be set')  
    return df

In [115]:
Customer_details='''


  --Take LAST day SNAPSHOT FOR active customers -- remove the eligibilty criteria --AND use it AS contract start feature
  
DECLARE
  start_dt_snpsht DATE DEFAULT '2022-09-01';
DECLARE
  end_dt_snpsht DATE DEFAULT '2022-11-30';
WITH
  date_sql AS (
  SELECT
    DATE_TRUNC(DATE_SUB(start_dt_snpsht, INTERVAL 3 month), month) AS eligibility_date ),
  ADC_Customer_Base AS (
  SELECT
    customer_id,
    dealer_customer_id,
    primary_login_id,
    dealer_name,
    DATE(join_date) AS join_date,
    account_type_name,
    customer_type_name,
    CASE
      WHEN account_type_name='Standalone' THEN 'Smart_Camera'
      WHEN account_type_name='Awareness and Automation' THEN 'Smart_Automation_Plus'
    ELSE
    'Monitored'
  END
    AS Package,
    DATE_DIFF(end_dt_snpsht,DATE(join_date), MONTH) AS Tenure_months
  FROM
    `cio-datahub-enterprise-pr-183a.src_adc.bq_customer_account_details`
  WHERE
    DATE(last_updt_ts) = end_dt_snpsht QUALIFY ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY last_updt_ts DESC) = 1
  ORDER BY
    join_date,
    customer_id ),
  Telus_customers AS (
  SELECT
    bacct_bus_bacct_num AS BAN,
    cust_bus_cust_id,
    DATE(pi_cntrct_start_ts) AS contract_start_date,
    DATE(pi_cntrct_end_ts) AS contract_end_date,
    CASE
      WHEN DATE(pi_cntrct_end_ts) = "9999-12-31" THEN 'contract_expired'
      WHEN DATE_DIFF(DATE(pi_cntrct_end_ts),end_dt_snpsht, DAY) < 0 THEN 'contract_expired'
      WHEN DATE_DIFF(DATE(pi_cntrct_end_ts),end_dt_snpsht, DAY) <= 90 THEN 'contract_expiring_within_90_days'
      WHEN DATE_DIFF(DATE(pi_cntrct_end_ts),end_dt_snpsht, DAY) > 90 THEN 'contract_expiring_after_90_days'
    ELSE
    'Undefined_contract_end_date'
  END
    AS contract_end_status,
    DATE_DIFF(end_dt_snpsht,DATE(pi_cntrct_start_ts), MONTH) AS Contract_start_months
  FROM
    `cio-datahub-enterprise-pr-183a.ent_cust_cust.bq_prod_instnc_snpsht`
  WHERE
    DATE(prod_instnc_ts) = end_dt_snpsht
    AND pi_prod_instnc_typ_cd ='SMHM' #Serice type
    AND bus_prod_instnc_src_id = 1001 #BANs that are FOR home services
    AND pi_prod_instnc_stat_cd IN ('A')
    AND consldt_cust_typ_cd = 'R' QUALIFY ROW_NUMBER() OVER (PARTITION BY cust_bus_cust_id ORDER BY prod_instnc_ts DESC) = 1
  ORDER BY
    cust_bus_cust_id ),
  Merge_data AS (
  SELECT
    *
  FROM
    ADC_Customer_Base a
  INNER JOIN
    Telus_customers b
  ON
    a.dealer_customer_id=b.cust_bus_cust_id )
SELECT
  *
FROM
  Merge_data
  
  
'''

In [116]:
Customer_info=extract_bq_data(bq_client, sql=Customer_details)

In [117]:
Customer_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 342243 entries, 0 to 342242
Data columns (total 15 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   customer_id            342243 non-null  Int64 
 1   dealer_customer_id     342243 non-null  object
 2   primary_login_id       342243 non-null  Int64 
 3   dealer_name            342243 non-null  object
 4   join_date              342243 non-null  object
 5   account_type_name      342243 non-null  object
 6   customer_type_name     342243 non-null  object
 7   Package                342243 non-null  object
 8   Tenure_months          342243 non-null  Int64 
 9   BAN                    342243 non-null  Int64 
 10  cust_bus_cust_id       342243 non-null  object
 11  contract_start_date    309544 non-null  object
 12  contract_end_date      342243 non-null  object
 13  contract_end_status    342243 non-null  object
 14  Contract_start_months  309544 non-null  Int64 
dtype

In [118]:
Customer_info['join_date'].min()

datetime.date(2011, 8, 11)

In [119]:
Customer_info['contract_start_date'].value_counts()

2022-11-23    757
2022-11-24    756
2022-11-25    747
2022-11-17    694
2022-11-26    692
             ... 
2018-03-11      1
2014-08-14      1
2016-07-09      1
2009-12-18      1
2015-07-13      1
Name: contract_start_date, Length: 3495, dtype: int64

In [120]:
Customer_info['contract_end_status'].value_counts()

contract_expiring_after_90_days     271844
contract_expired                     62303
contract_expiring_within_90_days      8096
Name: contract_end_status, dtype: int64

In [121]:
Customer_info['dealer_name'].value_counts()

TELUS Communications Inc.        341920
ADT by TELUS                        277
TELUS Pre-Production                 38
TELUS Custom Security Systems         8
Name: dealer_name, dtype: int64

In [122]:
Customer_info['Package'].value_counts()

Monitored                206628
Smart_Automation_Plus     89533
Smart_Camera              46082
Name: Package, dtype: int64

In [123]:
BAN_list=Customer_info['BAN'].value_counts().rename_axis('BAN').reset_index(name='unique_counts')

In [124]:
BAN_list_1=BAN_list[BAN_list.unique_counts==1]

In [125]:
BAN_list_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 334134 entries, 3866 to 337999
Data columns (total 2 columns):
 #   Column         Non-Null Count   Dtype
---  ------         --------------   -----
 0   BAN            334134 non-null  Int64
 1   unique_counts  334134 non-null  Int64
dtypes: Int64(2)
memory usage: 8.3 MB


In [126]:
Customer_info_1=Customer_info.merge(BAN_list_1[['BAN']],on='BAN',how='inner')

In [127]:
Customer_info_1['BAN'].value_counts().rename_axis('BAN').reset_index(name='unique_counts')

,BAN,unique_counts
0,604257832,1
1,603602524,1
2,603578571,1
3,603581088,1
4,603582220,1
...,...,...
334129,605535126,1
334130,604168543,1
334131,605473868,1
334132,605542416,1


In [128]:
Customer_info_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 334134 entries, 0 to 334133
Data columns (total 15 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   customer_id            334134 non-null  Int64 
 1   dealer_customer_id     334134 non-null  object
 2   primary_login_id       334134 non-null  Int64 
 3   dealer_name            334134 non-null  object
 4   join_date              334134 non-null  object
 5   account_type_name      334134 non-null  object
 6   customer_type_name     334134 non-null  object
 7   Package                334134 non-null  object
 8   Tenure_months          334134 non-null  Int64 
 9   BAN                    334134 non-null  Int64 
 10  cust_bus_cust_id       334134 non-null  object
 11  contract_start_date    302317 non-null  object
 12  contract_end_date      334134 non-null  object
 13  contract_end_status    334134 non-null  object
 14  Contract_start_months  302317 non-null  Int64 
dtype

In [129]:
Customer_info_1.head()

,customer_id,dealer_customer_id,primary_login_id,dealer_name,join_date,account_type_name,customer_type_name,Package,Tenure_months,BAN,cust_bus_cust_id,contract_start_date,contract_end_date,contract_end_status,Contract_start_months
0,2749224,100703810,3023307,TELUS Communications Inc.,2014-03-27,Security System,Customer,Monitored,104,604257832,100703810,2020-06-08,2023-06-08,contract_expiring_after_90_days,29
1,3074563,100717850,3403099,TELUS Communications Inc.,2014-09-03,Security System,Customer,Monitored,98,604262115,100717850,2017-01-01,2020-01-01,contract_expired,70
2,4658846,98349886,5270571,TELUS Communications Inc.,2016-09-06,Security System,Customer,Monitored,74,603598022,98349886,None,9999-12-31,contract_expired,<NA>
3,4666777,98350897,5279863,TELUS Communications Inc.,2016-09-13,Security System,Customer,Monitored,74,603598154,98350897,2019-11-03,2022-11-03,contract_expired,36
4,4674146,98330901,5288927,TELUS Communications Inc.,2016-09-13,Security System,Customer,Monitored,74,603578571,98330901,2016-09-13,2019-09-13,contract_expired,74


In [130]:
Customer_info_1['Contract_start_months'].value_counts()

0      15394
3      15033
1      14410
2      13974
4      13788
       ...  
311        1
304        1
267        1
214        1
271        1
Name: Contract_start_months, Length: 293, dtype: Int64

In [131]:
Customer_info_1['Tenure_months'].value_counts()

0      18433
3      16952
1      16698
2      15719
4      15489
       ...  
83         1
93         1
105        1
79         1
121        1
Name: Tenure_months, Length: 114, dtype: Int64

In [132]:
Customer_info_1[Customer_info_1.Tenure_months==0]

,customer_id,dealer_customer_id,primary_login_id,dealer_name,join_date,account_type_name,customer_type_name,Package,Tenure_months,BAN,cust_bus_cust_id,contract_start_date,contract_end_date,contract_end_status,Contract_start_months
2855,14472524,1071038,17104929,TELUS Communications Inc.,2022-11-04,Security System,Customer,Monitored,0,208632193,1071038,None,9999-12-31,contract_expired,<NA>
2913,14531946,95307845,17178070,TELUS Communications Inc.,2022-11-01,Awareness and Automation,Customer,Smart_Automation_Plus,0,603039506,95307845,2022-09-13,2025-09-13,contract_expiring_after_90_days,2
2957,14584236,101144411,17241906,TELUS Communications Inc.,2022-11-02,Awareness and Automation,Customer,Smart_Automation_Plus,0,604367428,101144411,2022-09-21,2025-09-21,contract_expiring_after_90_days,2
3029,14670210,91600889,17346332,TELUS Communications Inc.,2022-11-05,Awareness and Automation,Customer,Smart_Automation_Plus,0,602527218,91600889,None,9999-12-31,contract_expired,<NA>
3047,14692710,19795959,17373288,TELUS Communications Inc.,2022-11-25,Awareness and Automation,Customer,Smart_Automation_Plus,0,225112927,19795959,2022-10-08,2025-10-08,contract_expiring_after_90_days,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
334129,14990858,105057670,17736469,TELUS Communications Inc.,2022-11-28,Security System,Customer,Monitored,0,605525455,105057670,2022-11-28,2025-11-28,contract_expiring_after_90_days,0
334130,14990967,105057332,17736599,TELUS Communications Inc.,2022-11-28,Security System,Customer,Monitored,0,605525366,105057332,2022-11-28,2025-11-28,contract_expiring_after_90_days,0
334131,14991150,104936104,17736837,TELUS Communications Inc.,2022-11-28,Security System,Customer,Monitored,0,605471981,104936104,2022-11-29,2025-11-29,contract_expiring_after_90_days,0
334132,14992770,97744201,17738978,TELUS Communications Inc.,2022-11-28,Security System,Customer,Monitored,0,605551516,97744201,2022-11-29,2027-11-29,contract_expiring_after_90_days,0


In [133]:
config= bigquery.job.LoadJobConfig()

# config._properties['timePartitioning'] = {'field': 'Month_Year'}
config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE

Table_BQ = 'SHS.SHS_churn_model_base_data_sep_nov_2022'

bq_table_instance= bq_client.load_table_from_dataframe(Customer_info_1, Table_BQ,job_config=config)